# Basics - Setting everything up

Open this repository in a GitHub Codespace.
Before you start with anything else, make sure you setup the infrastructure required. Follow the readme file in the root folder to do this!

To start with the workshop, if not already done, or opened in a Devcontainer run this in the top level folder:

```
pip install -r requirements.txt
```




# Working with the Azure OpenAI API library

In this notebook, we will explore how to use OpenAI's language models (LLMs) to generate responses to various types of questions. The code in the next cell sets up the necessary environment by importing required modules, loading environment variables, and initializing the Azure OpenAI client. This setup is crucial for making API calls to OpenAI's services.

In [ ]:
import os
from openai import OpenAI, AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
    
# To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings. 
# Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens

model = os.getenv("AZURE_OPENAI_COMPLETION_MODEL") if os.getenv("AZURE_OPENAI_COMPLETION_MODEL") is not None else "gpt-4o-mini"

if os.environ.get("AZURE_OPENAI_API_KEY"):
    print("Using Azure OpenAI")
    client = AzureOpenAI(
        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    )
else:
    print("Using Inference API")
    base_url="https://models.inference.ai.azure.com"
    api_key=os.environ["GITHUB_TOKEN"]

    client = OpenAI(
        base_url=base_url,
        api_key=api_key,
    )

To see if stuff works in general, you can run this.

In [ ]:
response = client.chat.completions.create(
    model = model,    
    messages = [{"role" : "assistant", "content" : "The one thing I love more than anything else is "}],
)

print(response.choices[0].message.content)

This is the object model for receiving questions.

In [ ]:
from enum import Enum
from pydantic import BaseModel

class QuestionType(str, Enum):
    multiple_choice = "multiple_choice"
    true_or_false = "true_or_false"
    estimation = "estimation"

class Ask(BaseModel):
    question: str | None = None
    type: QuestionType
    correlationToken: str | None = None

class Answer(BaseModel):
    answer: str
    correlationToken: str | None = None
    promptTokensUsed: int | None = None
    completionTokensUsed: int | None = None


# Your Mission: 
Adjust the function below. Ensure the answers provided are correct and in a short precise format.



In [ ]:

async def ask_question(ask: Ask):
    # """
    # # Ask a question
    # """

    # Send a completion call to generate an answer
    print('Sending a request to openai')
    
    start_phrase =  ask.question
    response: openai.types.chat.chat_completion.ChatCompletion = None
    
    response = client.chat.completions.create(
        model = model,
        messages = [{"role" : "assistant", "content" : start_phrase}, 
                     { "role" : "system", "content" : "Answer this question:"}]
    )

    print(response.choices[0].message.content)
    print(response)

    answer = Answer(answer=response.choices[0].message.content)
    answer.correlationToken = ask.correlationToken
    answer.promptTokensUsed = response.usage.prompt_tokens
    answer.completionTokensUsed = response.usage.completion_tokens

    return answer

Use this snippet to try your method with several questions.

In [ ]:

# For this kind of question, the model should only answer with the letter of the correct answer
ask = Ask(question="Which of the following is a color? a. Tree b. Flower c. Green", type=QuestionType.multiple_choice)
answer = await ask_question(ask)
print('Answer:', answer)

What about other types of questions?

In [ ]:
# For this kind of question, the model should only answer with "True" or "False"
ask = Ask(question="Is the sky blue?", type=QuestionType.true_or_false)
answer = await ask_question(ask)
print('Answer:', answer)

In [ ]:
# For this kind of question, the model should only answer with a number
ask = Ask(question="How many stars are there in the universe?", type=QuestionType.estimation)
answer = await ask_question(ask)
print('Answer:', answer)

# Summary
In this notebook, we have explored how to use OpenAI's language models to generate responses to various types of questions. We have set up the necessary environment, loaded environment variables, and initialized the Azure OpenAI client. We have also defined a function to generate answers to questions and tested it with various examples. This demonstrates the power of LLMs in providing accurate and contextually relevant information.

# Up Next
In the next lab we'll begin looking at AI orchestrators. Whereas the OpenAI library simplifies working with the OpenAI API, orchestrators take things to the next level!